In [5]:
# ✅ INSTALL DEPENDENCIES
!pip install evaluate
!pip install optuna
!pip install -U transformers

In [6]:
# ✅ IMPORTS
import os
import re
import wandb
import optuna
import evaluate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, AutoConfig,
    TrainingArguments, Trainer, EarlyStoppingCallback,
    default_data_collator
)
from transformers.integrations import WandbCallback
from datasets import Dataset
from datetime import datetime

from optuna import trial


In [7]:
# ✅ MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: chen-benm (chen-benm-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [9]:
# ✅ CONSTANTS
MODEL_NAME_TWITTER = "google/electra-base-discriminator"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Optional: safer SDPA fallback on some Colab combos
if torch.cuda.is_available():
    torch.backends.cuda.sdp_kernel(enable_flash=False, enable_mem_efficient=False, enable_math=True)


/usr/lib/python3.12/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)


In [10]:
# ✅ LOAD & CLEAN DATA
label2id = {
    "Extremely Negative": 0,
    "Negative": 1,
    "Neutral": 2,
    "Positive": 3,
    "Extremely Positive": 4
}

train_path = "/content/drive/MyDrive/תואר שני/deep/Corona_NLP_train_clean.xls" #Change location where you saved the cleaned data
test_path = "/content/drive/MyDrive/תואר שני/deep/Corona_NLP_test_clean.xls" #Change location where you saved the cleaned data

train_df = pd.read_csv(train_path, encoding="ISO-8859-1")
test_df  = pd.read_csv(test_path,  encoding="ISO-8859-1")

train_df["label"] = train_df["Sentiment"].map(label2id)
test_df["label"]  = test_df["Sentiment"].map(label2id)

train_df = train_df[["cleaned_tweets","label"]].dropna(subset=["cleaned_tweets"])
test_df  = test_df[["cleaned_tweets","label"]].dropna(subset=["cleaned_tweets"])

def normalize_tweet(t: str) -> str:
    t = str(t)
    t = re.sub(r"http\S+", "<url>", t)
    t = re.sub(r"@\w+", "<user>", t)
    t = re.sub(r"\d+", "<number>", t)
    return t.strip()

for df in (train_df, test_df):
    df["cleaned_tweets"] = df["cleaned_tweets"].astype(str).map(normalize_tweet)
    df.query("cleaned_tweets.str.len() >= 3", engine="python", inplace=True)

train_df["label"] = train_df["label"].astype(int)
test_df["label"]  = test_df["label"].astype(int)

# Split
train_df, val_df = train_test_split(
    train_df, test_size=0.2, stratify=train_df["label"], random_state=42
)
test_df_final = test_df.copy()

# Hard checks
assert {"cleaned_tweets","label"}.issubset(train_df.columns)
assert train_df["label"].between(0,4).all() and val_df["label"].between(0,4).all()


In [11]:
# ✅ TOKENIZER + SPECIAL TOKENS (ADD ONCE, OUTSIDE OBJECTIVE)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_TWITTER, use_fast=True)

# Ensure pad token (usually present for RoBERTa; safe to check)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": "<pad>"})

# Add your custom specials only if missing
custom_specials = ["<url>", "<user>", "<number>"]
to_add = [t for t in custom_specials if t not in tokenizer.get_vocab()]
num_added = 0
if to_add:
    num_added = tokenizer.add_special_tokens({"additional_special_tokens": to_add})

print(f"Added {num_added} new tokens: {to_add if to_add else '[]'}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Added 3 new tokens: ['<url>', '<user>', '<number>']


In [12]:
# ✅ DATASETS
train_dataset_hf = Dataset.from_pandas(train_df.reset_index(drop=True))
val_dataset_hf = Dataset.from_pandas(val_df.reset_index(drop=True))
test_dataset_hf = Dataset.from_pandas(test_df_final.reset_index(drop=True))

In [13]:
# ✅ METRICS
acc_metric = evaluate.load("accuracy")
f1_metric  = evaluate.load("f1")
prec_metric = evaluate.load("precision")
rec_metric  = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        # use macro F1 to match metric_for_best_model
        "f1_macro": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
        # keep weighted variants if you like viewing them
        "precision_weighted": prec_metric.compute(predictions=preds, references=labels, average="weighted")["precision"],
        "recall_weighted": rec_metric.compute(predictions=preds, references=labels, average="weighted")["recall"],
    }

In [14]:
# ✅ MODEL FACTORY (RESIZES EMBEDDINGS TO MATCH TOKENIZER)
def build_model(dropout: float | None = None):
    config = AutoConfig.from_pretrained(
        MODEL_NAME_TWITTER,
        num_labels=5,
        problem_type="single_label_classification",
        id2label={0:"Extremely Negative",1:"Negative",2:"Neutral",3:"Positive",4:"Extremely Positive"},
        label2id={"Extremely Negative":0,"Negative":1,"Neutral":2,"Positive":3,"Extremely Positive":4},
        # If dropout is provided from Optuna, apply it to all relevant fields
        hidden_dropout_prob=dropout if dropout is not None else None,
        attention_probs_dropout_prob=dropout if dropout is not None else None,
        classifier_dropout=dropout if dropout is not None else None,
    )
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME_TWITTER, config=config)
    # keep tokenizer/model in sync (because you added <url>/<user>/<number>)
    model.resize_token_embeddings(len(tokenizer))
    if tokenizer.pad_token_id is not None:
        model.config.pad_token_id = tokenizer.pad_token_id
    return model

In [15]:
# ✅ OPTUNA OBJECTIVE — logs to Weights & Biases per trial
def objective(trial):
    import wandb

    # ---- Hyperparams to search (your requested space)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 3e-5, log=True)
    max_length    = trial.suggest_categorical("max_length", [96, 128])
    epochs        = trial.suggest_int("epochs", 3, 4)
    dropout       = trial.suggest_float("dropout", 0.08, 0.18)
    patience = trial.suggest_int("patience", 2, 4)
    batch_size = trial.suggest_categorical("batch_size", [16, 32])
    weight_decay = trial.suggest_categorical("weight_decay", [0.0, 0.01, 0.02])
   # warmup_ratio = trial.suggest_categorical("warmup_ratio", [0.05, 0.1])

    # ---- Start a fresh W&B run for this trial
    run = wandb.init(
        project="hf-electra",
        name=f"trial-{trial.number}",
        reinit=True,
        config={
            "learning_rate": learning_rate,
            "max_length": max_length,
            "batch_size": batch_size,
            "patience": patience,
            "epochs": epochs,
            "weight_decay": weight_decay,
            #"warmup_ratio": warmup_ratio,
            "dropout": dropout,
            "model_name": MODEL_NAME_TWITTER,
        },
    )

    try:
        # ---- Tokenize for this trial's max_length
        def preprocess(examples):
            enc = tokenizer(
                examples["cleaned_tweets"],
                truncation=True,
                padding="max_length",
                max_length=max_length,
            )
            enc["labels"] = examples["label"]
            return enc

        print(f"[DEBUG] Trial #{trial.number} → max_length = {max_length}")


        train_tok = train_dataset_hf.map(preprocess, batched=True, remove_columns=train_dataset_hf.column_names)
        val_tok   = val_dataset_hf.map(preprocess,   batched=True, remove_columns=val_dataset_hf.column_names)
        train_tok.set_format(type="torch")
        val_tok.set_format(type="torch")

        # ---- Build model (now receives dropout)
        model = build_model(dropout=dropout)

        # ---- Training args with W&B reporting
        args = TrainingArguments(
            output_dir=f"./hf_roberta_optuna_FINAL/{trial.number}",
            run_name=f"trial-{trial.number}",        # W&B run name
            report_to=["wandb"],                     # enable W&B logging

            learning_rate=learning_rate,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=epochs,
            weight_decay=weight_decay,

            #FIX: correct argument name
            eval_strategy="steps",
            save_strategy="steps",
            eval_steps=300,
            save_steps=300,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            greater_is_better=True,

            save_total_limit=1,                      # keep disk usage tiny
            logging_steps=100,
            seed=42,
            fp16=torch.cuda.is_available(),
        )

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=train_tok,
            eval_dataset=val_tok,
            tokenizer=tokenizer,
            data_collator=default_data_collator,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=patience)],
        )

        trainer.train()
        metrics = trainer.evaluate()

        # record best checkpoint dir for this trial
        best_ckpt = trainer.state.best_model_checkpoint
        trial.set_user_attr("best_checkpoint", best_ckpt)

        # ensure best model (already loaded) is saved in output_dir
        trainer.save_model()

        # Log final eval metrics explicitly too
        wandb.log({
            "final_eval/accuracy": metrics.get("eval_accuracy"),
            "final_eval/f1_macro": metrics.get("eval_f1_macro"),
        })

        # 🔧 Return the SAME metric used for model selection
        return metrics["eval_accuracy"]

    finally:
        # Ensure the run is closed even if an error occurs
        wandb.finish()

In [16]:
# ✅ RUN OPTUNA
study = optuna.create_study(direction="maximize", study_name="hf-robertatwitter-attempt3")
study.optimize(objective, n_trials=10)
print("Best value:", study.best_value)
print("Best params:", study.best_params)

[I 2025-08-21 08:38:24,770] A new study created in memory with name: hf-robertatwitter-attempt3
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[DEBUG] Trial #0 → max_length = 96


Map:   0%|          | 0/32100 [00:00<?, ? examples/s]

Map:   0%|          | 0/8026 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3436758371.py:83: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Weighted,Recall Weighted
300,1.493000,1.400614,0.362322,0.312060,0.446581,0.362322
600,1.118500,1.181964,0.492275,0.508049,0.532700,0.492275
900,0.997000,1.011993,0.593571,0.605991,0.602937,0.593571
1200,0.914400,1.095688,0.566035,0.579269,0.589143,0.566035
1500,0.833800,0.934011,0.648019,0.659615,0.654451,0.648019
1800,0.834700,0.996795,0.617618,0.628404,0.642013,0.617618
2100,0.761500,0.887439,0.681410,0.691679,0.686522,0.681410
2400,0.772000,0.942976,0.657862,0.666786,0.671166,0.657862
2700,0.726500,0.905586,0.675928,0.685064,0.684685,0.675928
3000,0.733400,0.959002,0.654373,0.664248,0.668479,0.654373


eval/accuracy,▁▄▆▅▇▇█▇█▇█
eval/f1_macro,▁▅▆▆▇▇███▇█
eval/loss,█▅▃▄▂▂▁▂▁▂▁
eval/precision_weighted,▁▄▆▅▇▇███▇█
eval/recall_weighted,▁▄▆▅▇▇█▇█▇█
eval/runtime,█▁▂▃▂▁▃▄▃▄▃
eval/samples_per_second,▁█▇▆▇█▆▅▆▅▆
eval/steps_per_second,▁█▇▆▇█▆▅▆▅▆
final_eval/accuracy,▁
final_eval/f1_macro,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████


[I 2025-08-21 09:53:56,154] Trial 9 finished with value: 0.6814104161475205 and parameters: {'learning_rate': 2.3213404368784804e-05, 'max_length': 96, 'epochs': 4, 'dropout': 0.1773852280121806, 'patience': 3, 'batch_size': 32, 'weight_decay': 0.0}. Best is trial 1 with value: 0.722651383005233.


Best value: 0.722651383005233
Best params: {'learning_rate': 2.795687090878637e-05, 'max_length': 96, 'epochs': 3, 'dropout': 0.12261239119691239, 'patience': 4, 'batch_size': 16, 'weight_decay': 0.0}


In [18]:
from shutil import copytree
from pathlib import Path

best_trial = study.best_trial
best_dir = best_trial.user_attrs.get("best_checkpoint", None)
assert best_dir is not None, "No best_checkpoint found on the best trial."

FINAL_DIR = Path("./best_model_autosaved")
copytree(best_dir, FINAL_DIR, dirs_exist_ok=True)
print("✅ Best model copied to:", FINAL_DIR)

✅ Best model copied to: best_model_autosaved


Compression techniques

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os, torch, copy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import nn
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, classification_report

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Point to your saved checkpoint folder (where config.json + pytorch_model.bin live)
BASE_DIR = "./best_model_autosaved"

model = AutoModelForSequenceClassification.from_pretrained(BASE_DIR).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(BASE_DIR)

In [20]:
def evaluate_on(df, model, tokenizer, max_length=192, batch_size=64, device=DEVICE, desc="Eval"):
    class TweetDataset(torch.utils.data.Dataset):
        def __init__(self, dataframe, tokenizer, max_length=128):
            self.texts = dataframe["cleaned_tweets"].tolist()
            self.labels = dataframe["label"].tolist()
            self.tokenizer = tokenizer
            self.max_length = max_length
        def __len__(self): return len(self.texts)
        def __getitem__(self, idx):
            enc = self.tokenizer(
                str(self.texts[idx]),
                truncation=True,
                padding="max_length",
                max_length=self.max_length,
                return_tensors="pt",
            )
            return {
                "input_ids": enc["input_ids"][0],
                "attention_mask": enc["attention_mask"][0],
                "labels": torch.tensor(self.labels[idx], dtype=torch.long),
            }

    ds = TweetDataset(test_df_final, tokenizer, max_length=max_length)
    dl = DataLoader(ds, batch_size=batch_size, pin_memory=True)

    model = model.to(device).eval()
    all_y, all_p = [], []
    with torch.no_grad():
        for b in dl:
            ids = b["input_ids"].to(device)
            att = b["attention_mask"].to(device)
            y   = b["labels"].to(device)
            logits = model(ids, attention_mask=att).logits
            p = logits.argmax(dim=1)
            all_y.extend(y.cpu().numpy().tolist())
            all_p.extend(p.cpu().numpy().tolist())

    print(f"{desc} accuracy:", accuracy_score(all_y, all_p))
    print(classification_report(all_y, all_p, digits=4))

In [21]:
def compressed_models(base_model):
    compressed = {}

    # 1) Dynamic Quantization (CPU-only module)
    cpu_model = copy.deepcopy(base_model).to("cpu")
    qmodel = torch.quantization.quantize_dynamic(
        cpu_model,
        {nn.Linear},
        dtype=torch.qint8
    )
    qmodel = qmodel.eval()  # ✅ Set to eval mode
    compressed["quantized_cpu"] = qmodel

    # 2) Pruning (unstructured L1 across Linear layers)
    pruned = copy.deepcopy(base_model).to(DEVICE)
    params_to_prune = []
    for m in pruned.modules():
        if isinstance(m, nn.Linear):
            params_to_prune.append((m, "weight"))

    if len(params_to_prune) > 0:
        prune.global_unstructured(
            params_to_prune,
            pruning_method=prune.L1Unstructured,
            amount=0.40,  # 40% sparsity
        )
        # Remove reparametrization to bake in the zeroed-out weights
        for (m, _) in params_to_prune:
            try:
                prune.remove(m, "weight")
            except Exception:
                pass
    pruned = pruned.eval()  # ✅ Set to eval mode
    compressed["pruned"] = pruned

    # 3) FP16 (good for GPU inference)
    half_model = copy.deepcopy(base_model).half().to(DEVICE)
    half_model = half_model.eval()  # ✅ Set to eval mode
    compressed["fp16"] = half_model

    return compressed

In [22]:
# Base FP32
evaluate_on(test_df_final, model, tokenizer, max_length=192, desc="Base FP32")

# Get the compressed models dictionary
compressed_models_dict = compressed_models(model)

# FP16 (GPU)
evaluate_on(test_df_final, compressed_models_dict["fp16"], tokenizer, max_length=192, desc="FP16")

# Pruned (GPU)
evaluate_on(test_df_final, compressed_models_dict["pruned"], tokenizer, max_length=192, desc="Pruned")

# Quantized (CPU)
evaluate_on(test_df_final, compressed_models_dict["quantized_cpu"], tokenizer, max_length=192, device="cpu", desc="Quantized CPU")

Base FP32 accuracy: 0.6930979978925185
              precision    recall  f1-score   support

           0     0.7057    0.7534    0.7288       592
           1     0.6351    0.6638    0.6491      1041
           2     0.7632    0.7472    0.7551       617
           3     0.6625    0.6199    0.6405       947
           4     0.7611    0.7446    0.7527       599

    accuracy                         0.6931      3796
   macro avg     0.7055    0.7058    0.7052      3796
weighted avg     0.6937    0.6931    0.6930      3796



/tmp/ipython-input-874486186.py:6: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  qmodel = torch.quantization.quantize_dynamic(


FP16 accuracy: 0.6936248682824026
              precision    recall  f1-score   support

           0     0.7068    0.7534    0.7294       592
           1     0.6364    0.6657    0.6507      1041
           2     0.7645    0.7472    0.7557       617
           3     0.6625    0.6199    0.6405       947
           4     0.7598    0.7446    0.7521       599

    accuracy                         0.6936      3796
   macro avg     0.7060    0.7061    0.7057      3796
weighted avg     0.6942    0.6936    0.6935      3796

Pruned accuracy: 0.5956269757639621
              precision    recall  f1-score   support

           0     0.8317    0.4257    0.5631       592
           1     0.5154    0.8367    0.6379      1041
           2     0.7418    0.6937    0.7169       617
           3     0.5251    0.5966    0.5586       947
           4     0.9667    0.2421    0.3872       599

    accuracy                         0.5956      3796
   macro avg     0.7161    0.5589    0.5727      3796
weighte

In [23]:
OUT_DIR = "/content/drive/MyDrive/תואר שני/deep" #write the output directory here
os.makedirs(OUT_DIR, exist_ok=True)

# A) Save FP16
fp16_dir = os.path.join(OUT_DIR, "fp16")
os.makedirs(fp16_dir, exist_ok=True)
compressed_models_dict["fp16"].save_pretrained(fp16_dir)
tokenizer.save_pretrained(fp16_dir)

# B) Save pruned (after prune.remove, save_pretrained works)
pruned_dir = os.path.join(OUT_DIR, "pruned")
os.makedirs(pruned_dir, exist_ok=True)
compressed_models_dict["pruned"].save_pretrained(pruned_dir)
tokenizer.save_pretrained(pruned_dir)

# C) Save quantized CPU (state_dict + a tiny loader script)
q_dir = os.path.join(OUT_DIR, "quantized_cpu")
os.makedirs(q_dir, exist_ok=True)
torch.save(compressed_models_dict["quantized_cpu"].state_dict(), os.path.join(q_dir, "quantized_state_dict.pt"))

# Save a small loader so future-you can reload easily
with open(os.path.join(q_dir, "load_quantized.py"), "w") as f:
    f.write(
        "import torch\n"
        "from torch import nn\n"
        "from transformers import AutoModelForSequenceClassification\n"
        "def load_quantized(model_dir, state_path):\n"
        "    model = AutoModelForSequenceClassification.from_pretrained(model_dir)\n"
        "    model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)\n"
        "    sd = torch.load(state_path, map_location='cpu')\n"
        "    model.load_state_dict(sd, strict=False)\n"
        "    model.eval()\n"
        "    return model\n"
    )

print("Saved to:", OUT_DIR)

Saved to: /content/drive/MyDrive/תואר שני/deep
